# Lab. 10: Linear Programming

## Introduction

#### <u>The goal of this lab is to use Linear Programming to solve some problems.</u>

In [ ]:
from typing import Optional

import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import linprog

In [ ]:
def plot_feasible_region(
    c: NDArray[np.float64],
    A: NDArray[np.float64],
    h: NDArray[np.float64],
    bounds: list[tuple[float, float]] = [(0, 1), (0, 1)],
    optimal_solution: Optional[NDArray[np.float64]] = None,
):
    xs: list[NDArray[np.float64]] = []
    ys: list[NDArray[np.float64]] = []
    _, ax = plt.subplots()
    for i in range(A.shape[0]):
        x = np.linspace(bounds[0][0], bounds[0][1])
        y = (h[i] - A[i, 0] * x) / A[i, 1]
        ax.plot(x, y)
        xs.append(x)
        ys.append(y)
    if optimal_solution is not None:
        ax.plot(*optimal_solution, "ro")
    ax.set_xlim(bounds[0][0], bounds[0][1])
    ax.set_ylim(bounds[1][0], bounds[1][1])

    # find all vertexes of the feasible region
    # vertexes = []
    # for i in range(len(xs)):
    #     for j in range(i + 1, len(xs)):
    #         x1, x2 = xs[i], xs[j]
    #         y1, y2 = ys[i], ys[j]
    #         for k in range(len(x1)):
    #             if y1[k] == y2[k]:
    #                 vertexes.append([x1[k], y1[k]])

    # if optimal_solution is not None:
    #     vertexes.append(optimal_solution)

    # vertexes = np.array(vertexes)
    # vertexes = vertexes[np.all(A @ vertexes.T <= h[:, None], axis=0)]
    # vertexes = vertexes[np.all(vertexes >= 0, axis=1)]
    # vertexes = vertexes[np.all(vertexes <= 1, axis=1)]

    # # plot the feasible region
    # ax.fill(*vertexes.T, "b", alpha=0.3)

    plt.show()

## Exercises

You will not be asked to implement parts of the linear programmign paradigm, but instead to transform the description of problems into a linear programming problem, first into the *canonical form* and then into the *slack form*, and solve them using the `scipy` [library](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html#scipy-optimize-linprog). Moreover, for exercises 1 and 2 plot the solution space and highlight the different components.

### Example exercise

A large factory makes tables and chairs. Each table returns a profit of 200 EUR and each chair a
profit of 100 EUR. Each table takes 1 unit of metal and 3 units of wood and each chair takes 2 units
of metal and 1 unit of wood. The factory has 600 units of metal and 900 units of wood.

Find the number of **chairs** and **tables** the factory needs to produce to **maximize their profit**, constrained to the **resources** it have.

---

*Canonical form:*

maximize  $2x_1 + x_2$

subject to

> $3x_1 + x_2 ≤ 9$

> $x_1 + 2x_2 ≤ 6$

> $x_1, x_2 ≥ 0$

In [ ]:
c = np.array([2, 1])
A = np.array([[3, 1], [1, 2]])
h = np.array([9, 6])
linprog(-c, A_ub=A, b_ub=h, bounds=[(0, None), (0, None)], method="highs")

*Slack form:*

maximize  $2x_1 + x_2$

subject to

> $3x_1 + x_2 + s_1 = 9$

> $x_1 + 2x_2 + s_2 = 6$

> $x_1, x_2, s_1, s_2 ≥ 0$

In [ ]:
c = np.array([-2, -1])
A = np.array([[3, 1], [1, 2]])
h = np.array([9, 6])
print(
    linprog(
        c,
        A_eq=A,
        b_eq=h,
        bounds=[(0, None), (0, None)],
        method="highs",
    )
)

In [ ]:
feasible_region = np.array([[0, 0], [3, 0], [2.4, 1.8], [0, 3]], dtype=np.float64)

plot_feasible_region(c, A, h, [(0, 10), (0, 10)], [2.4, 1.8])

### Exercise 1/3

A company makes two products (X and Y) using two machines (A and B). Each unit of X that is produced requires 50 minutes processing time on machine A and 30 minutes processing time on machine B. Each unit of Y that is produced requires 24 minutes processing time on machine A and 33 minutes processing time on machine B.

At the start of the current week there are 30 units of X and 90 units of Y in stock. Available processing time on machine A is forecast to be 40 hours and on machine B is forecast to be 35 hours.

The demand for X in the current week is forecast to be 75 units and for Y is forecast to be 95 units. Company policy is to maximise the combined sum of the units of X and the units of Y in stock at the end of the week.

---

#### TODO: transform this problem into its slack form and solve it + visualize the solution space with the different components

In [ ]:
# canonical form
c = np.array([1, 1])
A = np.array([[50, 24], [30, 33]])
b = np.array([2400, 2100])
res = linprog(-c, A_ub=A, b_ub=b, bounds=[(45, None), (5, None)], method="highs")
print(res)
plot_feasible_region(c, A, b, [(45, 80), (0, 30)], res["x"])

In [ ]:
# slack form
c = np.array([-1, -1, 0, 0])
b = np.array([2400, 2100])
A = np.array([[50, 24, 1, 0], [30, 33, 0, 1]])
res = linprog(
    c,
    A_eq=A,
    b_eq=b,
    bounds=[(45, None), (5, None), (0, None), (0, None)],
    method="highs",
)
print(res)
plot_feasible_region(c, A, b, [(45, 80), (0, 30)], res["x"][:2])

### Exercise 2/3

A factory manufactures chairs and tables, each requiring the use of three operations: Cutting, Assembly, and Finishing. The first operation can be used at most 40 hours; the second at most 42 hours; and the third at most 25 hours. A chair requires 1 hour of cutting, 2 hours of assembly, and 1 hour of finishing; a table needs 2 hours of cutting, 1 hour of assembly, and 1 hour of finishing. If the profit is 20 per unit for a chair and 30 for a table, how many units of each should be manufactured to maximize profit?

---

#### TODO: transform this problem into its slack form and solve it + visualize the solution space with the different components

In [ ]:
# canonical form
c = np.array([20, 30])
A = np.array([[1, 2], [2, 1], [1, 1]])
h = np.array([40, 42, 25])
res = linprog(-c, A_ub=A, b_ub=h, bounds=[(0, None), (0, None)], method="highs")
print(res)
plot_feasible_region(c, A, h, [(0, 20), (0, 20)], res["x"])

In [ ]:
# slack form
c = np.array([-20, -30, 0, 0, 0])
b = np.array([40, 42, 25])
A = np.array([[1, 2, 1, 0, 0], [2, 1, 0, 1, 0], [1, 1, 0, 0, 1]])
res = linprog(
    c,
    A_eq=A,
    b_eq=b,
    bounds=[(0, None), (0, None), (0, None), (0, None), (0, None)],
    method="highs",
)
print(res)
plot_feasible_region(c, A, b, [(0, 20), (0, 20)], res["x"][:2])

### Exercise 3/3

A mutual fund has $ \\$ 100,000 $ to be invested over a three year horizon.

Three investment options are available:

1. **Annuity:**  the fund can  pay a same amount of new capital at the beginning of each of three years and receive a payoff of 130% of **total capital** invested  at the end of the third year. Once the mutual fund decides to invest in this annuity, it has to keep investing in all subsequent  years in the three year horizon.  
2. **Bank account:** the fund can deposit any amount  into a bank at the beginning of each year and receive its capital plus 6% interest at the end of that year. In addition, the mutual fund is permitted to borrow no more than \$20,000 at the beginning of each year and is asked to pay back the amount borrowed plus 6% interest at the end of the year. The mutual fund can choose whether to deposit or borrow at the beginning of each year.  
3. **Corporate bond:** At the beginning of the second year, a  corporate bond becomes available.
  The fund can buy an amount
  that is no more than $ \\$ $50,000 of this bond at the beginning of the second year and  at the end of the third year receive a payout of 130% of the amount invested in the bond.  


The mutual fund’s objective is to maximize total payout that it owns at the end of the third year.

---

#### **HINT:**

Let  $ x_1 $ be the amount of put in the annuity, $ x_2, x_3, x_4 $ be  bank deposit balances at the beginning of the three years,  and $ x_5 $ be the amount invested  in the corporate bond.

When $ x_2, x_3, x_4 $ are negative, it means that  the mutual fund has borrowed from  bank.

The table below shows the mutual fund’s decision variables together with the timing protocol described above:

||Year 1|Year 2|Year 3|
|:-----------------------:|:-----------------------:|:-----------------------:|:-----------------------:|
|Annuity|$ x_1 $|$ x_1 $|$ x_1 $|
|Bank account|$ x_2 $|$ x_3 $|$ x_4 $|
|Corporate bond|0|$ x_5 $|0|

---

#### **TODO:** formulate this as a linear programming problem try transform this problem into its slack form and solve it

#### *We suggest you to try to do it by yourself or in small group in class, but, in case of need, the solution can be found [here](https://tools-techniques.quantecon.org/lp_intro.html#example-2-investment-problem)*

In [ ]:
# slack form
c = np.array([3.9, 0, 0, 1.06, 1.3])
b = np.array([100000, 0, 0])
A = np.array([[1, 1, 0, 0, 0], [1, -1.06, 1, 0, 1], [1, 0, -1.06, 1, 0]])
bounds = [(0, None), (-20000, None), (-20000, None), (-20000, None), (0, 50000)]
res = linprog(-c, A_eq=A, b_eq=b, bounds=bounds, method="highs")
print(res)